# 使用Embedding API
注：为了方便embedding api调用，应将密钥填入llm_universe下的.env文件，代码将自动读取并加载环境变量。
## 一、使用OpenAI API
GPT有封装好的接口，我们简单封装即可。目前GPT embedding mode有三种，性能如下所示：
|模型 | 每美元页数 | [MTEB](https://github.com/embeddings-benchmark/mteb)得分 | [MIRACL](https://github.com/project-miracl/miracl)得分|
| --- | --- | --- | --- |
|text-embedding-3-large|9,615|54.9|64.6|
|text-embedding-3-small|62,500|62.3|44.0|
|text-embedding-ada-002|12,500|61.0|31.4|
* MTEB得分为embedding model分类、聚类、配对等八个任务的平均得分。
* MIRACL得分为embedding model在检索任务上的平均得分。  

从以上三个embedding model我们可以看出`text-embedding-3-large`有最好的性能和最贵的价格，当我们搭建的应用需要更好的表现且成本充足的情况下可以使用；`text-embedding-3-small`有着较好的性能跟价格，当我们预算有限时可以选择该模型；而`text-embedding-ada-002`是OpenAI上一代的模型，无论在性能还是价格都不如及前两者，因此不推荐使用。

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv


# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 如果你需要通过代理端口访问，你需要如下配置
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

def openai_embedding(text: str, model: str=None):
    # 获取环境变量 OPENAI_API_KEY
    api_key=os.environ['OPENAI_API_KEY']
    client = OpenAI(api_key=api_key)

    # embedding model：'text-embedding-3-small', 'text-embedding-3-large', 'text-embedding-ada-002'
    if model == None:
        model="text-embedding-3-small"

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = openai_embedding(text='要生成 embedding 的输入文本，字符串形式。')

API返回的数据为`json`格式，除`object`向量类型外还有存放数据的`data`、embedding model 型号`model`以及本次 token 使用情况`usage`等数据，具体如下所示：
```json
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.006929283495992422,
        ... (省略)
        -4.547132266452536e-05,
      ],
    }
  ],
  "model": "text-embedding-3-small",
  "usage": {
    "prompt_tokens": 5,
    "total_tokens": 5
  }
}
```
我们可以调用response的object来获取embedding的类型。

In [2]:
print(f'返回的embedding类型为：{response.object}')

返回的embedding类型为：list


embedding存放在data中，我们可以查看embedding的长度及生成的embedding。

In [3]:
print(f'embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为：{response.data[0].embedding[:10]}')

embedding长度为：1536
embedding（前10）为：[0.03884002938866615, 0.013516489416360855, -0.0024250170681625605, -0.01655769906938076, 0.024130908772349358, -0.017382603138685226, 0.04206013306975365, 0.011498954147100449, -0.028245486319065094, -0.00674333656206727]


我们也可以查看此次embedding的模型及token使用情况。

In [4]:
print(f'本次embedding model为：{response.model}')
print(f'本次token使用情况为：{response.usage}')

本次embedding model为：text-embedding-3-small
本次token使用情况为：Usage(prompt_tokens=12, total_tokens=12)


## 二、使用文心千帆API
Embedding-V1是基于百度文心大模型技术的文本表示模型，Access token为调用接口的凭证，使用Embedding-V1时应先凭API Key、Secret Key获取Access token，再通过Access token调用接口来embedding text。同时千帆大模型平台还支持bge-large-zh等embedding model。

In [5]:
import requests
import json

def wenxin_embedding(text: str):
    # 获取环境变量 wenxin_api_key、wenxin_secret_key
    api_key = os.environ['QIANFAN_AK']
    secret_key = os.environ['QIANFAN_SK']

    # 使用API Key、Secret Key向https://aip.baidubce.com/oauth/2.0/token 获取Access token
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={0}&client_secret={1}".format(api_key, secret_key)
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    
    # 通过获取的Access token 来embedding text
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/embeddings/embedding-v1?access_token=" + str(response.json().get("access_token"))
    input = []
    input.append(text)
    payload = json.dumps({
        "input": input
    })
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)
# text应为List(string)
text = "要生成 embedding 的输入文本，字符串形式。"
response = wenxin_embedding(text=text)

Embedding-V1每次embedding除了有单独的id外，还有时间戳记录embedding的时间。

In [6]:
print('本次embedding id为：{}'.format(response['id']))
print('本次embedding产生时间戳为：{}'.format(response['created']))

本次embedding id为：as-hvbgfuk29u
本次embedding产生时间戳为：1711435238


同样的我们也可以从response中获取embedding的类型和embedding。

In [7]:
print('返回的embedding类型为:{}'.format(response['object']))
print('embedding长度为：{}'.format(len(response['data'][0]['embedding'])))
print('embedding（前10）为：{}'.format(response['data'][0]['embedding'][:10]))

返回的embedding类型为:embedding_list
embedding长度为：384
embedding（前10）为：[0.060567744076251984, 0.020958080887794495, 0.053234219551086426, 0.02243831567466259, -0.024505289271473885, -0.09820500761270523, 0.04375714063644409, -0.009092536754906178, -0.020122773945331573, 0.015808865427970886]


## 三、使用讯飞星火API

### 尚未开放

## 四、使用智谱API
智谱有封装好的SDK，我们调用即可。

In [4]:
from zhipuai import ZhipuAI
def zhipu_embedding(text: str):
    ZHIPUAI_API_KEY = ""  #原始的代码是没有这行代码的
    # api_key = os.environ['ZHIPUAI_API_KEY']
    api_key = ZHIPUAI_API_KEY
    client = ZhipuAI(api_key=api_key)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response

text = '要生成 embedding 的输入文本，字符串形式。'
response = zhipu_embedding(text=text)
print(response)

object='list' data=[Embedding(object='embedding', index=0, embedding=[0.017893229, 0.064432174, -0.009351327, 0.027082685, 0.0040648775, -0.05599671, -0.042226028, -0.030019397, -0.01632937, 0.067769825, 0.025368802, -0.009653148, -0.050464496, 0.026528299, 0.0023294394, 0.00625851, 0.007950371, -0.10619533, -0.0064624813, -0.006465767, -0.004465786, 0.027298773, 0.02956282, 0.037749898, 0.023569249, 0.02223137, -0.03409359, 0.025905661, -0.022260765, -0.066684246, 0.025628077, 0.01964144, 0.041692913, -0.013005985, 0.01469556, 0.0119359465, 0.031962708, -0.0051111905, 0.025309127, -0.03727085, 0.016857088, -0.024697784, 0.03798244, -0.025598718, -0.038381483, 0.03749951, 0.0835946, 0.014087594, -0.025747592, 0.0610262, -0.0027598634, 0.025468636, 0.011879007, 0.011536968, 0.012097709, -0.017410792, -0.006488134, -0.018455712, -0.021024939, -0.005449268, 0.027530728, 0.0014936096, 0.05468244, -0.0055377656, -0.03201199, -0.024840808, -0.033149272, 0.051240917, -0.01526203, -0.009134962

response为`zhipuai.types.embeddings.EmbeddingsResponded`类型，我们可以调用`object`、`data`、`model`、`usage`来查看response的embedding类型、embedding、embedding model及使用情况。

In [9]:
print(f'response类型为：{type(response)}')
print(f'embedding类型为：{response.object}')
print(f'生成embedding的model为：{response.model}')
print(f'生成的embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为: {response.data[0].embedding[:10]}')

response类型为：<class 'zhipuai.types.embeddings.EmbeddingsResponded'>
embedding类型为：list
生成embedding的model为：embedding-2
生成的embedding长度为：1024
embedding（前10）为: [0.017893229, 0.064432174, -0.009351327, 0.027082685, 0.0040648775, -0.05599671, -0.042226028, -0.030019397, -0.01632937, 0.067769825]
embedding（前10）为: object='embedding' index=0 embedding=[0.017893229, 0.064432174, -0.009351327, 0.027082685, 0.0040648775, -0.05599671, -0.042226028, -0.030019397, -0.01632937, 0.067769825, 0.025368802, -0.009653148, -0.050464496, 0.026528299, 0.0023294394, 0.00625851, 0.007950371, -0.10619533, -0.0064624813, -0.006465767, -0.004465786, 0.027298773, 0.02956282, 0.037749898, 0.023569249, 0.02223137, -0.03409359, 0.025905661, -0.022260765, -0.066684246, 0.025628077, 0.01964144, 0.041692913, -0.013005985, 0.01469556, 0.0119359465, 0.031962708, -0.0051111905, 0.025309127, -0.03727085, 0.016857088, -0.024697784, 0.03798244, -0.025598718, -0.038381483, 0.03749951, 0.0835946, 0.014087594, -0.025747592, 0.06102